In [ ]:
#===========================================================================
Rewrite IPOPT
===========================================================================#

using DataFrames
using Ipopt
include("ind_shnorm.jl");
include("ind_shnormMPEC.jl");
include("invertshares.jl");
include("jacobian.jl");
include("mksharesim.jl");
include("MPEC_constraint.jl");
include("MPEC_gradient.jl");
include("MPEC_hessian.jl");
include("MPEC_jacobian.jl");
include("MPEC_objective.jl");

########## Settings ##########

nn = 100;
tol_inner = 1.e-14;
prods = 25;
T = 50;
starts = 5;

### Number of products per market: Discrete uniform between 1 and prods ###
# Import fake data from Matlab
prodsMarket = readtable("MPEC_prodsMarket.csv",header=false);
prodsMarket = convert(Array,prodsMarket);

#=
Use two dimensional matrices for speed reasons (vectorization)
Market indices tell us the first and last spots of records for each market in
list of all products
=#
marketStarts = zeros(T,1)[:,1];
marketEnds = zeros(T,1)[:,1];
marketStarts[1] = 1;
marketEnds[1] = prodsMarket[1];
for t=2:T
  marketStarts[t] = marketEnds[t-1] + 1
  marketEnds[t] = marketStarts[t] + prodsMarket[t] - 1
end
marketStarts = round(Int,marketStarts);
marketEnds = round(Int,marketEnds);
numProdsTotal = marketEnds[T];   # Total number of products in all markets

########## True parameter values ##########

costparams = (ones(6,1)/2)[:1];
betatrue = [2; 1.5; 1.5; 0.5; -3];   # True mean tastes
betatrue0 = betatrue;
K = length(betatrue) - 1;    # Number of attributes
covrc = diagm([.5; .5; .5; .5; .2]);   # True covariances in tastes
rctrue = covrc[find(covrc)];
thetatrue = [betatrue; sqrt(rctrue)];
# Import fake data from Matlab
v = readtable("MPEC_v.csv",header=false);    # Draws for share integrals during estimation
v = convert(Array,v);
rc = chol(covrc)' * v;   # Draws for share integrals for data creation
sigmaxi = 0.5;
covX = -.8*ones(K-1,K-1)+1.8*eye(K-1);   # Allows for correlation in X's
covX[1,3] = .3;
covX[2,3] = .3;
covX[3,1] = .3;
covX[3,2] = .3;

########## Create indices to speed share calculations ##########

oo = ones(1,nn);   # Index for use in calculation of shares
sharesum = sparse(zeros(T,numProdsTotal)); # Used to create denominators in logit predicted shares (i.e. sums numerators)
for t=1:T
  sharesum[t,marketStarts[t]:marketEnds[t]] = 1
end
marketForProducts = zeros(numProdsTotal,1);    # Market indices for each product
# Used to expand one logit denominator per consumer to one for each product
for t=1:T
  marketForProducts[marketStarts[t]:marketEnds[t]] = t
end
marketForProducts = [Int(v) for v in marketForProducts];

############### Simulate data ###############

# Import fake data from Matlab
xi = readtable("MPEC_xi.csv",header=false);   # Draw demand shocks
xi = convert(Array,xi);
A = readtable("MPEC_A.csv",header=false);    # Product attributes
A = convert(Array,A);
prand = readtable("MPEC_prand.csv",header=false);
prand = convert(Array,prand);
price = 3 + xi*1.5 + prand + sum(A,2);
price = price[:,1];
z = readtable("MPEC_z.csv",header=false);
z = convert(Array,z);
x = [ones(numProdsTotal,1) A price];
share, nopurch = mksharesim(betatrue,x,xi,rc,prods,T,sharesum,marketForProducts);
y = log(share) - log(nopurch[marketForProducts,:]);   # Log-odds ratio for no RC logit estimation
# Matrix of instruments
iv = [ones(numProdsTotal,1) A z A.^2 A.^3 z.^2 z.^3 prod(A,2) prod(z,2) kron(A[:,1],ones(1,size(z,2))).*z kron(A[:,2],ones(1,size(z,2))).*z];
IV = iv;

########## 2SLS estimation of homogeneous logit model ##########

pz = iv*inv(iv'*iv)*iv';
xhat = pz*x;
PX2sls = inv(xhat'*xhat)*xhat';   # Projection matrix on weighted x-space for 2SLS
beta2sls = PX2sls*y;
beta2sls = beta2sls[:,1];
se2sls = sqrt(diag(mean((y-x*beta2sls).^2)*inv(x'*pz*x)));

#==========
Choices for GMM estimation
GMM weighting matrix follows Nevo
==========#

expmeanval0 = exp(y);   # Starting value for mean values
expmeanval0 = expmeanval0[:,1];
W = inv(IV'*IV);    # GMM weighting matrix
PX = inv(x'*IV*W*IV'*x)*x'*IV*W*IV';    # Used to calculate starting values

########## MPEC estimation of random coefficients logit model ##########

theta20 = 0.5*abs(beta2sls);
startvalues = readtable("MPEC_startvalues.csv",header=false);
startvalues = convert(Array,startvalues);
nIV = size(IV,2);   # Number of instrumental variables

GMPEC = 1.0e20;
CPUtMPEC = 0;
FuncEvalMPEC = 0;
GradEvalMPEC = 0;
HessEvalMPEC = 0;
expmeanval = expmeanval0;
X_MPEC = zeros(2*(K+1)+numProdsTotal+nIV, starts);
objective = zeros(starts,1);
statusMPEC = zeros(starts,1);
thetaMPEC1 = zeros(starts);
thetaMPEC2 = zeros(starts);
deltaMPEC = zeros(numProdsTotal);

for reps=1:starts
  # Note: start values set to give same initial GMM objective functional value as with NFP
  theta20 = startvalues[reps,:]';    # Starting values for standard deviations
  delta0 = invertshares(theta20,x,expmeanval,tol_inner,share,v,oo,sharesum,marketForProducts);   # Starting values for mean utilities using BLP inversion
  theta10 = PX*delta0;   # Starting values for linear parameters using IV like regression of delta on covariates
  resid0 = delta0 - x*theta10;   # Starting values for xi terms, not directly used
  g0 = IV'*resid0;   # Starting values for GMM moments
  x0 = [theta10; theta20; delta0; g0];   # Master starting values

  x_L = -Inf*ones(length(x0));   # Lower bounds for x
  x_L[K+2:2*K+2] = 0;    # Standard deviations are nonnegative
  x_L=vec(x_L);
  x_U = Inf*ones(length(x0));    # Upper bounds for x
  x_U=vec(x_U);

  nx0 = size(x0,1);    # Number of MPEC optimization parameters

  # Sparsity patterns of constraints, 0 if derivative 0, 1 otherwise
  # Derivatives of market shares
  c11 = zeros(numProdsTotal,K+1);    # Market shares with respect to mean parameters
  c12 = ones(numProdsTotal,K+1);   # Market shares with respect to standard deviations
  c13 = zeros(numProdsTotal,numProdsTotal);  # Market shares with respect to mean utilities
  for t=1:T
    c13[marketStarts[t]:marketEnds[t], marketStarts[t]:marketEnds[t]] = 1
  end
  c14 = zeros(numProdsTotal,nIV);    # Market shares with respect to moment values

  # Derivatives of moments
  c21 = ones(nIV,K+1);   # Moments with respect to mean parameters
  c22 = zeros(nIV,K+1);    # Moments with respect to standard deviations
  c23 = ones(nIV,numProdsTotal);   # Moments with respect to mean utilities
  c24 = eye(nIV);    # Moments with respect to moment values

  global ConsPattern = [c11 c12 c13 c14; c21 c22 c23 c24];

  # Hessian pattern
  HessianPattern = zeros(nx0,nx0);
  HessianPattern[2*K+3+numProdsTotal:end, 2*K+3+numProdsTotal:end] = ones(nIV,nIV);
  HessianPattern[K+2:2*K+2, K+2:2*K+2+numProdsTotal] = ones(K+1,K+1+numProdsTotal);
  HessianPattern[2*K+3:2*K+2+numProdsTotal, K+2:2*K+2] = ones(numProdsTotal,K+1);
  for t=1:T
    range = marketStarts[t]:marketEnds[t]
    index = 2*K+2+range
    HessianPattern[index,index] = ones(prodsMarket[t],prodsMarket[t])
  end
  global HessianPattern

  g_L = vec(zeros(size(ConsPattern,1),1));    # Lower bounds on constraints
  g_U = vec(zeros(size(ConsPattern,1),1));    # Upper bounds on constraints

  nonzeroJ = nnz(sparse(ConsPattern));           # Number of nonzeros in Jacobian
  HessianPattern_tril = tril(HessianPattern);
  nonzeroH = nnz(sparse(HessianPattern_tril));   # Number of nonzeros in Hessian

  prob = createProblem(nx0,x_L,x_U,size(ConsPattern,1),g_L,g_U,nonzeroJ,nonzeroH,
                      MPEC_objective,MPEC_constraint,MPEC_gradient,MPEC_jacobian,
                      MPEC_hessian);

  # prob = createProblem(
  #   nx0,                              # Number of variables
  #   lb,                              # Variable lower bounds
  #   ub,                              # Variable upper bounds
  #   size(ConsPattern,1),              # Number of constraints
  #   clb,                              # Constraint lower bounds
  #   cub,                              # Constraint upper bounds
  #   nonzeroJ,                         # Number of nonzeros in Jacobian
  #   nonzeroH,                         # Number of nonzeros in Hessian
  #   MPEC_objective,                   # Callback: objective function
  #   MPEC_constraint,                  # Callback: constraint evaluation
  #   MPEC_gradient,                    # Callback: objective function gradient
  #   MPEC_jacobian,                    # Callback: Jacobian evaluation
  #   MPEC_hessian)                     # Callback: Hessian evaluation

  addOption(prob,"jac_c_constant","no");
  addOption(prob,"hessian_constant","no");
  addOption(prob,"mu_strategy","adaptive");
  addOption(prob,"derivative_test","none");    # 'Second-order' 'none'
  addOption(prob,"tol",1e-6);
  addOption(prob,"max_iter",100);
  prob.x=vec(x0);

  status = solveProblem(prob);
  println(status);
  println(Ipopt.ApplicationReturnStatus[status]);
  println(prob.obj_val);
  X=prob.x;

  X_MPEC[:,reps] = X;
  objective[reps,1] = MPEC_objective(X);
  statusMPEC[reps,1] = status;

  theta1MPEC_rep = X[1:K+1];
  theta2MPEC_rep = X[K+2:2*K+2];
  deltaMPEC_rep = X[2*K+3:2*K+2+numProdsTotal,1];
  GMPEC_rep = MPEC_objective(X);
  INFOMPEC_rep = status;

  if GMPEC_rep < GMPEC && INFOMPEC_rep == 0
    thetaMPEC1 = theta1MPEC_rep
    thetaMPEC2 = theta2MPEC_rep
    deltaMPEC = deltaMPEC_rep
    GMPEC = GMPEC_rep
    INFOMPEC = INFOMPEC_rep
  end
end     # End loop over different start values

########## Covariance matrix for MPEC structural parameters ##########

deltaSE = invertshares(thetaMPEC2,x,expmeanval,tol_inner,share,v,oo,sharesum,marketForProducts);
resid = deltaSE - x*thetaMPEC1;     # xi
delta = deltaSE;
Ddelta = jacobian(thetaMPEC2,x,v,delta,K,numProdsTotal,prods,T,nn,prodsMarket,marketStarts,marketEnds,oo,sharesum,marketForProducts);
covg = zeros(size(IV,2),size(IV,2));
for ii = 1:length(IV[:,1])
    covg = covg + IV[ii,:]'*IV[ii,:]*(resid[ii]^2)
end
Dg = [x Ddelta]'*IV;    # Gradients of moment conditions
covMPEC = inv(Dg*W*Dg')*Dg*W*covg*W*Dg'*inv(Dg*W*Dg');
